# Practica 4
## Captcha Destroyer hecho por: Python Haters
#### - Hugo Vivanco Fernandez
#### - Jaime Isar Muñoz
#### - Daniel Lafuente Bazo
#### - Óscar Fabián Pineda Germán



## Parte 0: Tratamiento de imagenes
Trabajamos con .png y tenemos que ser capaces de procesarlos

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.models import Model
from PIL import Image
import numpy as np
import string
import os


symbols = string.ascii_lowercase + string.digits # All symbols captcha can contain
directorio_input = 'samples/'
directorio_output = 'samples_transform/'
CAPTCHA_LENGTH = 5
contador = 0


def getTargets():

    file_list = [f for f in os.listdir(directorio_input) if f.endswith('.png')]
    n = len(file_list)

    # Array para almacenar las etiquetas
    letersImg_array = np.zeros((n, CAPTCHA_LENGTH, len(symbols)))  # Shape (n, 5, 36)

    for i, pic in enumerate(file_list):
        pic_target = pic[:-4]  # Quitar '.png'
        
        # Solo procesar captchas de exactamente 5 caracteres
        if len(pic_target) == CAPTCHA_LENGTH:
            for j, k in enumerate(pic_target):
                indexSymbol = symbols.find(k.upper())
                if indexSymbol != -1:  # Solo si encuentra el símbolo
                    letersImg_array[i, j, indexSymbol] = 1

    # Ahora separar por cada letra para que keras lo acepte como 5 salidas
    targets_split = [letersImg_array[:, i, :] for i in range(CAPTCHA_LENGTH)]

    return targets_split  # Devuelve lista: [y1, y2, y3, y4, y5]

def imagen_a_matriz(ruta_imagen, umbral=128):

    # Abre y convierte a escala de grises
    imagen = Image.open(ruta_imagen)
    
    # Aplica umbral para binarizar (blanco o negro)
    imagen_bn = imagen.point(lambda p: 255 if p > umbral else 0)
    
    ancho, alto = imagen_bn.size
    pixeles = imagen_bn.load()
    
    matriz = []
    for y in range(alto):
        fila = []
        for x in range(ancho):
            pixel = pixeles[x, y]
            valor = 1 if pixel == 0 else 0  # Negro -> 1, Blanco -> 0
            fila.append(valor)
        matriz.append(fila)
    
    return matriz

def convertir_imagen(ruta_input, ruta_output, dimension=(50, 200)):

    # Abrir imagen
    imagen = Image.open(ruta_input)
    
    # Cambiar tamaño
    imagen = imagen.resize(dimension)
    
    # Convertir a blanco y negro
    imagen_bn = imagen.convert('L')  # 'L' es modo de 8 bits en escala de grises
    
    # Guardar la nueva imagen
    imagen_bn.save(ruta_output)

def mostrar_matriz(matriz, ancho=200, alto=50):
    for c in range(0, alto):
        for f in range(0, ancho):
            print(matriz[c][f], end="")
        print('\n')
    print('\n')

def createmodel(size=(50, 200, 1)):

    # Bloque de entrada
    inputs = layers.Input(shape=size)

    # Primer bloque
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)  # 32 filtros
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Segundo bloque
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Tercer bloque
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)

    outputs = []
    for _ in range(5):
        outputs.append(layers.Dense(36, activation='softmax')(x))

    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model



# Rellenar la variable 'datos' con las imagenes en binario
#datos = []
#for nombre_archivo in os.listdir(directorio_input):

#    Convertir todas las imagenes del directorio a blanco y negro y dimension establecidos
#    archivo_original = directorio_input + nombre_archivo
#    archivo_nuevo = directorio_output + nombre_archivo
#    convertir_imagen(archivo_original, archivo_nuevo)
#    datos.append( imagen_a_matriz(archivo_nuevo) )


# Almacenar todos los datos de entrada
datos = []
test = 3
for archivo in os.listdir(directorio_input):

    if(test == 0):
        break

    archivo_nuevo = directorio_output + archivo
    datos.append( imagen_a_matriz(archivo_nuevo) )
    test = test - 1
    contador = contador + 1


# Crear modelo
modelo = createmodel()
modelo.summary()

# Todos los datos para el entrenamiento
x = datos
y = getTargets()
y1, y2, y3, y4, y5 = y

# División
split_index = 970
X_train, X_test = x[:split_index], x[split_index:]
y1_train, y1_test = y1[:split_index], y1[split_index:]
y2_train, y2_test = y2[:split_index], y2[split_index:]
y3_train, y3_test = y3[:split_index], y3[split_index:]
y4_train, y4_test = y4[:split_index], y4[split_index:]
y5_train, y5_test = y5[:split_index], y5[split_index:]

# Compilacion del modelo
modelo.compile(
    loss=['categorical_crossentropy'] * 5,
    optimizer='adam',
    metrics=['accuracy'] * 5
)

X_train = np.array(X_train)
targets = [y1_train, y2_train, y3_train, y4_train, y5_train]

# Paso 3: Lanza el entrenamiento de forma segura
hist = modelo.fit(
    X_train,
    targets,
    batch_size=32,
    epochs=60,
    validation_split=0.2
)


Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20      │ (None, 50, 200,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_60 (Conv2D)  │ (None, 50, 200,   │        320 │ input_layer_20[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_60    │ (None, 25, 100,   │          0 │ conv2d_60[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_61 (Conv2D)  │ (None, 25, 100,   │     18,496 │ max_pooling2d_60… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_61    │ (None, 12, 50,    │          0 │ conv2d_61[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_62 (Conv2D)  │ (None, 12, 50,    │     73,856 │ max_pooling2d_61… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_62    │ (None, 6, 25,     │          0 │ conv2d_62[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_20          │ (None, 19200)     │          0 │ max_pooling2d_62… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_120 (Dense)   │ (None, 512)       │  9,830,912 │ flatten_20[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_121 (Dense)   │ (None, 36)        │     18,468 │ dense_120[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_122 (Dense)   │ (None, 36)        │     18,468 │ dense_120[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_123 (Dense)   │ (None, 36)        │     18,468 │ dense_120[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_124 (Dense)   │ (None, 36)        │     18,468 │ dense_120[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_125 (Dense)   │ (None, 36)        │     18,468 │ dense_120[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,015,924 (38.21 MB)

 Trainable params: 10,015,924 (38.21 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - dense_121_accuracy: 0.0000e+00 - dense_121_loss: 3.5274 - dense_122_accuracy: 0.0000e+00 - dense_122_loss: 3.5559 - dense_123_accuracy: 0.0000e+00 - dense_123_loss: 1.7969 - dense_124_accuracy: 0.0000e+00 - dense_124_loss: 1.8291 - dense_125_accuracy: 0.0000e+00 - dense_125_loss: 0.0000e+00 - loss: 10.7093 - val_dense_121_accuracy: 1.0000 - val_dense_121_loss: 0.6031 - val_dense_122_accuracy: 0.0000e+00 - val_dense_122_loss: 4.2430 - val_dense_123_accuracy: 0.0000e+00 - val_dense_123_loss: 3.9301 - val_dense_124_accuracy: 0.0000e+00 - val_dense_124_loss: 3.6736 - val_dense_125_accuracy: 0.0000e+00 - val_dense_125_loss: 0.0000e+00 - val_loss: 12.4498
Epoch 2/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - dense_121_accuracy: 1.0000 - dense_121_loss: 0.4635 - dense_122_accuracy: 1.0000 - dense_122_loss: 0.7012 - dense_123_accuracy: 0.5000 - dense_123_loss: 0.8380 - dense_124_accuracy: 0.5000 - dense_124_loss: 1.0802 - dense_125_accuracy: 0.000

## Parte 1: Reconocer un dígito o letra deformado

Para empezar el proyecto decidimos ir paso a paso. Primero identificaremos numeros y letras de forma individual.

## Parte 2: Reconocer una cadena de dígitos y letras bien separadas
Ahora que sabemos identificar letras y números por separado podemos agruparlos bien separados en una cadena para leerlos y escribirlos en orden de aparición

## Parte 3: Reconocer dígitos y letras que se juntan/mezclan
Por último queda ser capaces de distinguir los números y letras que están solapados en la cadena para poder escribirlos